# Praca domowa 3 - Raport

W ramach pracy domowej muszę wytrenować trzy dowolne klasyfikatory i sprawdzić ich skuteczność na [zbiorze danych](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package)
Na początku trzeba wczytać dane oraz sprawdzić jakie wartości przyjmują kolejne parametry.

In [1]:
using IJulia 
IJulia.installkernel("Julia nodeps")
IJulia.clear_output();

In [2]:
using Pkg # Wczytanie pakietów
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("StatsBase")
Pkg.add("Random")
Pkg.add("MLLabelUtils")
Pkg.add("DataFramesMeta")
Pkg.add("DecisionTree")
Pkg.add("ScikitLearn")
Pkg.add("MLBase")
Pkg.build("PyCall")
IJulia.clear_output();

## Wczytanie danych
Do tego wykorzystujemy pakiet CSV i od razu konwertujemy go do formatu DataFrames.
Poprawność wczytania widać poniżej.
Należy również usuąć jedną kolumne ponieważ zawierała inforamcje o ilości opadów następnego dnia przez co ta kolumna zawierała informacje o tym czy jutro będzie padać.
Taka informacja trywializuje nasze zadanie i powoduje że inne zmienne nie będą miały wpływu na model.
Usuniemy również te kolumny które nie były podane w treści zadania

In [3]:
using CSV, DataFrames
df1 = CSV.read(joinpath(Pkg.dir("DataFrames"), "C:\\Users\\lukas\\Desktop\\weatherAUS.csv"));
df = df1[:,:]
df = select(df, Not([:RISK_MM,:Date,:Location,:WindGustDir,:WindDir9am,:WindDir3pm]))
IJulia.clear_output();
first(df,5)

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am
,String,String,String,String,String,String,String
1,13.4,22.9,0.6,NA,NA,44,20
2,7.4,25.1,0,NA,NA,44,4
3,12.9,25.7,0,NA,NA,46,19
4,9.2,28,0,NA,NA,24,11
5,17.5,32.3,1,NA,NA,41,7


Spójrzmy teraz na to jakie typy mają wszystkie kolumnu oraz jakie wartośći przyjmuje pierwszy rząd z naszej ramki danych. 

In [4]:
show(first(df,1), allrows=false, allcols=true)

1×18 DataFrame
│ Row │ MinTemp │ MaxTemp │ Rainfall │ Evaporation │ Sunshine │ WindGustSpeed │
│     │ String  │ String  │ String   │ String      │ String   │ String        │
├─────┼─────────┼─────────┼──────────┼─────────────┼──────────┼───────────────┤
│ 1   │ 13.4    │ 22.9    │ 0.6      │ NA          │ NA       │ 44            │

│ Row │ WindSpeed9am │ WindSpeed3pm │ Humidity9am │ Humidity3pm │ Pressure9am │
│     │ String       │ String       │ String      │ String      │ String      │
├─────┼──────────────┼──────────────┼─────────────┼─────────────┼─────────────┤
│ 1   │ 20           │ 24           │ 71          │ 22          │ 1007.7      │

│ Row │ Pressure3pm │ Cloud9am │ Cloud3pm │ Temp9am │ Temp3pm │ RainToday │
│     │ String      │ String   │ String   │ String  │ String  │ String    │
├─────┼─────────────┼──────────┼──────────┼─────────┼─────────┼───────────┤
│ 1   │ 1007.1      │ 8        │ NA       │ 16.9    │ 21.8    │ No        │

│ Row │ RainTomorrow │
│     │ String 

Większość wartości jest typu String.
sprawdźmy czy kolumny przyjmują dużo unikalnych wartośći.
Jeśli tak to oznacza że mają one wartośći numeryczne.
Natomiast jeżeli jest ich kilka to zapewne są to wartości nominalne.

In [5]:
values = [string(size(unique(c))[1])*" : " for c in eachcol(df)]
n =size(values)[1]
for i in 1:n
    
    values[i] = values[i]*String.(names(df))[i]
end
values

18-element Array{String,1}:
 "390 : MinTemp"
 "506 : MaxTemp"
 "680 : Rainfall"
 "357 : Evaporation"
 "146 : Sunshine"
 "68 : WindGustSpeed"
 "44 : WindSpeed9am"
 "45 : WindSpeed3pm"
 "102 : Humidity9am"
 "102 : Humidity3pm"
 "547 : Pressure9am"
 "550 : Pressure3pm"
 "11 : Cloud9am"
 "11 : Cloud3pm"
 "441 : Temp9am"
 "501 : Temp3pm"
 "3 : RainToday"
 "2 : RainTomorrow"

Możemy zobaczyć że tylko dwie kolumny mają bardzo mało różnych wartości.
Ostatnia z nich jest naszą kolumną celu.
Mówi ona czy następnego dnia padało.
To będzie celem do przewidzenia dla naszych modeli.
Drugą kolumną jest wartość czy w danym dniu padało.

In [6]:
unique(df.RainToday)

3-element Array{String,1}:
 "No"
 "Yes"
 "NA"

In [7]:
a = size(df[([i in ["No"] for i in df.RainToday] .> 0),:])[1]
b = size(df[([i in ["Yes"] for i in df.RainToday] .> 0),:])[1]
c = size(df[([i in ["NA"] for i in df.RainToday] .> 0),:])[1]
println("Ilość rekordów gdzie nie padało danego dnia : ",a)
println("Ilość rekordów gdzie padało danego dnia : ",b)
println("Ilość rekordów gdzie nie ma danych czy padało danego dnia : ",c )

Ilość rekordów gdzie nie padało danego dnia : 109332
Ilość rekordów gdzie padało danego dnia : 31455
Ilość rekordów gdzie nie ma danych czy padało danego dnia : 1406


Z tego widać że większość rekordów jest dobrze przygotowana.
Z tego powodu zdecydowałem się usunąć 1 % dziwnych wartośći.

In [8]:
using DataFramesMeta
df = @transform(df, RainToday = replace(:RainToday, "NA" => missing))
df = @transform(df, RainToday = replace(:RainToday, "Yes" => "1"))
df = @transform(df, RainToday = replace(:RainToday, "No" => "0"))
df = dropmissing(df, :RainToday)
c = size(df[([i in ["NA"] for i in df.RainToday] .> 0),:])[1]
size(df)

(140787, 18)

Sprawdźmy teraz ile kolumn z naszej ramki danych zawiera wartości NA.

In [9]:
vec =  [count(x->x=="NA",unique(c)) for c in eachcol(df)]
vec = [string(c) for c in vec]
n =size(vec)[1]
for i in 1:n    
    vec[i] = vec[i]*" : "*String.(names(df))[i]
end
vec

18-element Array{String,1}:
 "1 : MinTemp"
 "1 : MaxTemp"
 "0 : Rainfall"
 "1 : Evaporation"
 "1 : Sunshine"
 "1 : WindGustSpeed"
 "1 : WindSpeed9am"
 "1 : WindSpeed3pm"
 "1 : Humidity9am"
 "1 : Humidity3pm"
 "1 : Pressure9am"
 "1 : Pressure3pm"
 "1 : Cloud9am"
 "1 : Cloud3pm"
 "1 : Temp9am"
 "1 : Temp3pm"
 "0 : RainToday"
 "0 : RainTomorrow"

Jak widać tylko 3 kolumny nie mają braków danych.
Ilość opadów deszczu, wartość binarna czy w danym dniu padało oraz to czy w następnym dniu padało.
(To jest nasza szukana wartość)

In [10]:
df = @transform(df, WindGustSpeed = replace(:WindGustSpeed, "NA" => "0"))
df = @transform(df, WindSpeed9am = replace(:WindSpeed9am, "NA" => "0"))
df = @transform(df, WindSpeed3pm = replace(:WindSpeed3pm, "NA" => "0"))
df = @transform(df, Evaporation = replace(:Evaporation, "NA" => "0"))
df = @transform(df, Sunshine = replace(:Sunshine, "NA" => "0"));

Postanowiłem zamienić zmienne "NA" na 0 w przypadku prędkości wiatru, miary odparowywania i czasu nasłoniecznienia.
Wynika to z tego że brak danych jest zapewne spowodowany brakiem wiatru,odparowywania i słońca w danym dniu.

In [11]:
df = @transform(df, Humidity9am = replace(:Humidity9am, "NA" => missing))
df = @transform(df, Humidity3pm = replace(:Humidity3pm, "NA" => missing))
df = @transform(df, Pressure9am = replace(:Pressure9am, "NA" => missing))
df = @transform(df, Pressure3pm = replace(:Pressure3pm, "NA" => missing))
df = @transform(df, Cloud9am = replace(:Cloud9am, "NA" => missing))
df = @transform(df, Cloud3pm = replace(:Cloud3pm, "NA" => missing))
df = dropmissing(df, :Humidity9am)
df = dropmissing(df, :Humidity3pm)
df = dropmissing(df, :Pressure9am)
df = dropmissing(df, :Pressure3pm)
df = dropmissing(df, :Cloud9am)
df = dropmissing(df, :Cloud3pm)
df = @transform(df, RainTomorrow = replace(:RainTomorrow, "No" => "0"))
df = @transform(df, RainTomorrow = replace(:RainTomorrow, "Yes" => "1"));

Dla zmiennych wilgotnośći, zachmurzenia i ciśnienia założyłem że brak wspisu oznacza niepełność danych oraz ich hipotetyczną niedokładność.
Dlatego z powodu dużej ilości danych usunąłem rzędy z brakującymi informacjami.

Przyporządkowałem również wartości binarne dla zmiennych celu.
"0" dla braku deszczu i "1" dla jego występowania.

In [12]:
print("Wielkość naszej ramki danych po usunięciu części danych ")
printstyled(size(df)[1],bold = true)
print(".")
first(df,5)

Wielkość naszej ramki danych po usunięciu części danych 77618.

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am
,String,String,String,String,String,String,String
1,17.5,32.3,1,0,0,41,7
2,15.9,21.7,2.2,0,0,31,15
3,15.9,18.6,15.6,0,0,61,28
4,14.1,20.9,0,0,0,22,11
5,13.5,22.9,16.8,0,0,63,6


In [13]:
print(size(df))

(77618, 18)

Została nam kolumna z minimalną oraz maksymalną temperaturą.
Policzmy ile jest rekordów z brakującymi danami.

In [14]:
a = size(df[([i in ["NA"] for i in df.MinTemp] .> 0),:])[1]
b = size(df[([i in ["NA"] for i in df.MaxTemp] .> 0),:])[1]
println("Ilość rekordów z nie podaną maksymalną temperaturą : ",b)
println("Ilość rekordów z nie padaną minimalną temperaturą : ",a)

Ilość rekordów z nie podaną maksymalną temperaturą : 29
Ilość rekordów z nie padaną minimalną temperaturą : 55


Widać że jest ich mało więc dla uproszczenia usuniemy te rekordy.

In [15]:
println("Ilość rekordów przed usunięciem NA :",size(df)[1])
df = @transform(df, MinTemp = replace(:MinTemp, "NA" => missing))
df = @transform(df, MaxTemp = replace(:MaxTemp, "NA" => missing))
df = dropmissing(df, :MinTemp)
df = dropmissing(df, :MaxTemp)
println("Ilość rekordów po usunięciu NA :",size(df)[1])

Ilość rekordów przed usunięciem NA :77618
Ilość rekordów po usunięciu NA :77534


In [16]:
vec =  [count(x->x=="NA",unique(c)) for c in eachcol(df)]
vec = [string(c) for c in vec]
n =size(vec)[1]
for i in 1:n    
    vec[i] = vec[i]*" : "*String.(names(df))[i]
end
print(string(vec))

["0 : MinTemp", "0 : MaxTemp", "0 : Rainfall", "0 : Evaporation", "0 : Sunshine", "0 : WindGustSpeed", "0 : WindSpeed9am", "0 : WindSpeed3pm", "0 : Humidity9am", "0 : Humidity3pm", "0 : Pressure9am", "0 : Pressure3pm", "0 : Cloud9am", "0 : Cloud3pm", "0 : Temp9am", "0 : Temp3pm", "0 : RainToday", "0 : RainTomorrow"]

Nie mamy już żadnych braków danych.

Sprawdzmy teraz jakie typ danych mają kolumny w ramce danych.
Wszystkie dane są numeryczne więc powinny być w typie Float64.

In [17]:
eltype.(eachcol(df))

18-element Array{DataType,1}:
 String
 String
 String
 String
 String
 String
 String
 String
 String
 String
 String
 String
 String
 String
 String
 String
 String
 String

Skonwertujmy teraz te dane na numeryczne wartośći

In [18]:
df[:MinTemp] = [parse(Float64,x) for x in df[:MinTemp]]
df[:MaxTemp] = [parse(Float64,x) for x in df[:MaxTemp]] 
df[:Rainfall] = [parse(Float64,x) for x in df[:Rainfall]]
df[:Sunshine] = [parse(Float64,x) for x in df[:Sunshine]] 
df[:Evaporation] = [parse(Float64,x) for x in df[:Evaporation]] 
df[:WindGustSpeed] = [parse(Float64,x) for x in df[:WindGustSpeed]] 
df[:WindSpeed9am] = [parse(Float64,x) for x in df[:WindSpeed9am]] 
df[:WindSpeed3pm] = [parse(Float64,x) for x in df[:WindSpeed3pm]] 
df[:Humidity9am] = [parse(Float64,x) for x in df[:Humidity9am]] 
df[:Humidity3pm] = [parse(Float64,x) for x in df[:Humidity3pm]] 
df[:Pressure9am] = [parse(Float64,x) for x in df[:Pressure9am]] 
df[:Pressure3pm] = [parse(Float64,x) for x in df[:Pressure3pm]] 
df[:Cloud9am] = [parse(Float64,x) for x in df[:Cloud9am]] 
df[:Cloud3pm] = [parse(Float64,x) for x in df[:Cloud3pm]] 
df[:Temp9am] = [parse(Float64,x) for x in df[:Temp9am]] 
df[:Temp3pm] = [parse(Float64,x) for x in df[:Temp3pm]] 
df[:RainToday] = [parse(Float64,x) for x in df[:RainToday]] 
df[:RainTomorrow] = [parse(Float64,x) for x in df[:RainTomorrow]] 
IJulia.clear_output();

In [19]:
using Random
df_train = deepcopy(df)
n = convert(Int16,floor(nrow(df)/10)) # Bierzemy 10% wartości do zbioru testowego,
                                      # reszte jako zbiór treningowy. 
vec = sort(randperm(nrow(df))[1:n])   
df_test = df[vec,:]
for i in 1:n
    deleterows!(df_train,vec[i] - i + 1)
end

Mamy podział na zbiór treningowy i testowy.
Podzielmy jeszcze nasze dane na zbiory X i Y oznaczające zmienne features i labels.

In [20]:
df_train_y = df_train[:RainTomorrow]
df_test_y = df_test[:RainTomorrow]
df_train_x = df_train[:, filter(x -> x != :RainTomorrow, names(df_train))]
df_test_x = df_test[:, filter(x -> x != :RainTomorrow, names(df_test))]
IJulia.clear_output();

# Klasyfikatory

Do tego zadania użyjemu klasyfikatorów z pakietu **DecisionTree**.
Jest to pakiet który implementuje algorytmy dzrewa decezyjnego (CART) oraz losowych lasów.
Ten pakiet implementuje te algorytmy w języku [Julia.](https://julialang.org/)
Sam pakiet znajduje się [tutaj](https://github.com/bensadeghi/DecisionTree.jl)

Najpierw stworzymy zwykłe dzrzewo decyzyjne z maksymalną głębokością oraz parametrem pruning = 10%.

In [21]:
using DecisionTree

model_dt = DecisionTreeClassifier(max_depth=30)

DecisionTreeClassifier
max_depth:                30
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  nothing
root:                     nothing

In [22]:
df_train_x=convert(Matrix,df_train_x) #Wszystkie modele na wejściu muszą mieć macierze.
df_test_x=convert(Matrix,df_test_x);

In [23]:
fit!(model_dt, df_train_x, df_train_y)

DecisionTreeClassifier
max_depth:                30
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [0.0, 1.0]
root:                     Decision Tree
Leaves: 8147
Depth:  30

In [24]:
using ScikitLearn.CrossValidation: cross_val_score
using MLBase
accuracy = cross_val_score(model_dt, df_test_x, reshape(df_test_y,(7753,1)),cv = 3)

3-element Array{Float64,1}:
 0.765183752417795
 0.75
 0.7786377708978328

## Random Forest

Teraz stworze model metodą random forset z tego samego pakietu.

In [25]:
model_f = build_forest(df_train_y, df_train_x, 2, 10, 0.5, 6)

Ensemble of Decision Trees
Trees:      10
Avg Leaves: 61.2
Avg Depth:  6.0

In [26]:
n_folds=3; n_subfeatures=2
accuracy = nfoldCV_forest(df_test_y, df_test_x, n_folds, n_subfeatures)


Fold 1
Mean Squared Error:     0.0745467854051952
Correlation Coeff:      0.7827692798893219
Coeff of Determination: 0.5828293002718226

Fold 2
Mean Squared Error:     0.06937034113779526
Correlation Coeff:      0.7986359342075682
Coeff of Determination: 0.6053419317238534

Fold 3
Mean Squared Error:     0.07254182267936414
Correlation Coeff:      0.7937063579644398
Coeff of Determination: 0.5977495869482081

Mean Coeff of Determination: 0.5953069396479614


3-element Array{Float64,1}:
 0.5828293002718226
 0.6053419317238534
 0.5977495869482081

## Prune Tree

Trzecim algorytmem będzie prune tree.
Jest to metoda by pozbyć się overfit'ingu.
Polega on na "wycinaniu" tych części dzrzewa która zawiera już mało informacji i ma słaby wpływ na klasyfikacje naszych danych.
Do tego używa się parametru odcięcia który podaje jako hiperparametr.

In [27]:
model_p = build_tree(df_train_y, df_train_x)
model = prune_tree(model_p, 0.9)

Decision Tree
Leaves: 3940
Depth:  27

In [28]:
n_folds=3
accuracy = nfoldCV_tree(df_test_y, df_test_x, n_folds)


Fold 1
Mean Squared Error:     0.04711264951887188
Correlation Coeff:      0.8568555475865995
Coeff of Determination: 0.7341903163653196

Fold 2
Mean Squared Error:     0.048882453139637926
Correlation Coeff:      0.8496858242357442
Coeff of Determination: 0.7219005382953505

Fold 3
Mean Squared Error:     0.04842050014450225
Correlation Coeff:      0.8565251404123752
Coeff of Determination: 0.7336057995723195

Mean Coeff of Determination: 0.7298988847443298


3-element Array{Float64,1}:
 0.7341903163653196
 0.7219005382953505
 0.7336057995723195

# Podumowanie i ocena algorytmów

Według walidacji krzyżowej najlepszy wynik ma "najprostrzy" algorytm.
Zbliżony wynik ma Prune Tree natomiast Random Forest "odstaje" od reszty. 
